In [127]:
#Question 1

'''Implement the template based chord recognition algorithm. Define a function that takes as input the
path to a wav file and returns the estimated chords sequence labels.
The output must be a list

λ_pred = [λ_pred0, λpred_1, ..., λpred_N−1] (1)

where each element λpred_n is the predicted chord label for the time frame n. 
The length of the list depends on the feature rate, i.e., both on the window length and hop size 
used for the chromagram computation and on the downsampling factor, if feature downsampling is 
applied.

The chord templates to be considered are the major triads and the minor triads, leading to a total of 24 templates.
Once the function is defined in the notebook, test the function on the wav file Beatles LetItBe.wav,
available in /data/wav/ folder, and print or plot the output.

Explain in the report the idea behind the template-based chord recognition algorithm and detail each
step implemented in the code, including pre processing and post processing phases.
'''
#Import all packages

import os, sys
import math
import sys
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import librosa
import IPython.display as ipd
%matplotlib inline

def generate_template_matrix(templates):
    template_matrix = np.zeros((12, 12 * templates.shape[1]))
    for shift in range(12):
        #np.roll: roll array elements along a given axis.
        template_matrix[:, shift::12] = np.roll(templates, shift, axis=0)
    return template_matrix

def normalize_feature_sequence(X, norm='2', threshold=0.0001, v=None):
    """Normalizes the columns of a feature sequence

    Args:
        X: Feature sequence
        norm: The norm to be applied. '1', '2', 'max'
        threshold: An+ threshold below which the vector `v` used instead of normalization
        v: Used instead of normalization below `threshold`. If None, uses unit vector for given norm

    Returns:
        X_norm: Normalized feature sequence
    """
    K, N = X.shape
    X_norm = np.zeros((K, N))
    if norm == '1':
        if v is None:
            v = np.ones(K, dtype=np.float64) / K
        for n in range(N):
            s = np.sum(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v

    if norm == '2':
        if v is None:
            v = np.ones(K, dtype=np.float64) / np.sqrt(K)
        for n in range(N):
            s = np.sqrt(np.sum(X[:, n] ** 2))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v
                
    if norm == 'max':
        if v is None:
            v = np.ones(K)
        for n in range(N):
            s = np.max(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v
    return X_norm

def smooth_downsample_feature_sequence(X, Fs, filt_len=41, down_sampling=10, w_type='boxcar'):
    """
    Args:
        X: Feature sequence
        Fs: Frame rate of `X`
        filt_len: Length of smoothing filter
        down_sampling: Downsampling factor
        w_type: Window type of smoothing filter

    Returns:
        X_smooth: Smoothed and downsampled feature sequence
        Fs_feature: Frame rate of `X_smooth`
    """
    # use expand dims to add one dimension to the window, from (L, ) to (1,L) 
    filt_kernel = np.expand_dims(signal.get_window(w_type, filt_len), axis=0)
    X_smooth = signal.convolve(X, filt_kernel, mode='same') / filt_len
    X_smooth = X_smooth[:, ::down_sampling]
    Fs_feature = Fs / down_sampling
    return X_smooth, Fs_feature

def analysis_template_match(chromagram, templates, smoothing_window_length=None, smoothing_down_sampling=None,
                            Fs=22050, norm_chromagram='2', norm_output='2'):
    
    chroma_normalized = normalize_feature_sequence(chromagram, norm=norm_chromagram)
    if smoothing_window_length and smoothing_down_sampling:
        chroma_normalized, Fs_feat = smooth_downsample_feature_sequence(chroma_normalized, 
                                                                        down_sampling=smoothing_down_sampling,
                                                                        filt_len=smoothing_window_length, Fs=Fs)
    templates_normalized = normalize_feature_sequence(templates, norm=norm_chromagram)
    chord_similarity = np.matmul(templates_normalized.T, chroma_normalized)
    if norm_output:
         chord_similarity = normalize_feature_sequence(chord_similarity, norm=norm_output)
    
    chord_max = (chord_similarity == chord_similarity.max(axis=0)).astype(int)
    return chord_similarity, chord_max

def chordList(templates, template_sequence):
    template_sequence[template_sequence != 0] = 1 #evidenzia bene le note nella matrice in uscita
    chords = [] #inizializza la lista degli accordi
    
    #analizza ogni campione della matrice dell'audio e di quella di riferimento mettendoli a confronto
    #quando i campioni corrispondono assegna il nome dell'accordo all'array chords
    for i in range(template_sequence.shape[1]):
        for j in range(templates.shape[1]):
            if(np.array_equal(template_sequence[:,i], templates[:,j])): 
                #majorChords
                if(j==0):
                    chords.append("C")
                if(j==1):
                    chords.append("C#")
                if(j==2):
                    chords.append("D")
                if(j==3):
                    chords.append("D#")
                if(j==4):
                    chords.append("E")
                if(j==5):
                    chords.append("F")
                if(j==6):
                    chords.append("F#")
                if(j==7):
                    chords.append("G")
                if(j==8):
                    chords.append("G#")
                if(j==9):
                    chords.append("A")
                if(j==10):
                    chords.append("A#")   
                if(j==11):
                    chords.append("B")
                    
                #minorChords
                if(j==12):
                    chords.append("Cm")
                if(j==13):
                    chords.append("C#m")
                if(j==14):
                    chords.append("Dm")
                if(j==15):
                    chords.append("D#m")
                if(j==16):
                    chords.append("Em")
                if(j==17):
                    chords.append("Fm")
                if(j==18):
                    chords.append("F#m")
                if(j==19):
                    chords.append("Gm")
                if(j==20):
                    chords.append("G#m")
                if(j==21):
                    chords.append("Am")
                if(j==22):
                    chords.append("A#m")   
                if(j==23):
                    chords.append("Bm")
    return chords

#funzione che prende in ingresso un path di un segnale audio
def template_based_chord_recognigtion(audioPath):   
    #Generate Chord Template Matrix
    template_cmaj = np.array([[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]]).T
    template_cmin = np.array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]]).T
    templates = generate_template_matrix(np.concatenate((template_cmaj, template_cmin), axis=1))
  
    #Parameters
    Fs = 22050
    N = 4096
    H = 2048
    
    #Read audio
    x, Fs = librosa.load(audioPath, sr=Fs)
    
    #Algorithm
    X = librosa.stft(x, n_fft=N, hop_length=H, pad_mode='constant', center=True)
    X = np.abs(X) ** 2
    
    #logarithmic compression
    gamma = 0.1
    X = np.log(1 + gamma * np.abs(X) ** 2)
    
    #chromagram
    C = librosa.feature.chroma_stft(S=X, sr=Fs, tuning=0, hop_length=H, n_fft=N, norm=None)
    
    #analize chords
    chords_sim, chords_max = analysis_template_match(C, templates, 
                                                 smoothing_window_length=None,
                                                 smoothing_down_sampling=None, 
                                                 Fs=Fs,
                                                 norm_chromagram='2', 
                                                 norm_output='max')
    # Compute normalized binary templates of analysis
    templates_normalized = normalize_feature_sequence(templates, norm='2')
    # by multipling the most probable chords by the templates (normalized), you obtain the sequence of chords chroma
    # by chroma
    
    #matrice contentente gli array degli accordi in binario
    template_sequence = np.matmul(templates_normalized, chords_max)
    #crea la matrice degli accordi in stringhe
    chords = chordList(templates, template_sequence)
    
    return chords

#Execute function
audioWav = os.path.join('data','wav', 'Beatles_LetItBe.wav')#read the path
chords_WAV = template_based_chord_recognigtion(audioWav)#ritorna gli accordi
print(chords_WAV, len(chords_WAV))


['G#', 'G', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'C', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'C', 'C', 'C', 'C', 'Am', 'F', 'F', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'F', 'F', 'Dm', 'Dm', 'Dm', 'Am', 'Dm', 'Dm', 'Dm', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'Dm', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Em', 'Em', 'Em', 'Em', 'Em', 'Dm', 'Dm', 'Dm', 'Dm', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Cm', 'G', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Am', 'G#', 'Cm', 'C', 'C', 'C', 'C', 'C', 'Cm', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'C', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'Am', 'A', 'D', 'A', 'Am', 'C#m', 'E', 'E', 'E', 'E', 'E', 'E'

In [125]:
#Question 2
'''
Write a function to load and preprocess a reference annotation (or ground truth) file, saved in CSV
format. The function should take as input the path to a CSV file and produce as output a list of ground
truth chord labels, after suitable pre processing. The output must be a list

λgt = [λgt0, λgt1, ..., λgtN−1] (2)

where each element λgtn is the ground truth chord label for the time window n. 
The length of the list must be adapted to match the the feature rate.

The reference annotations stored in the CSV file are given in the form of labelled segments, each
specified as a triplet (start, end, λ) where start and end are expressed in seconds. To load the CSV file
check the csv library (distributed with Python) or Pandas library (needs to be installed).

In the preprocessing step you should
• convert the segment-based annotation into a frame-based label sequence adapted to the feature rate
used for the chroma sequence;
• convert the labels used in the annotation file to match the chord labels used for the chord recognition
algorithm in terms of enharmonic equivalence (i.e., Db = C# );

• reduce the chord label set used in the annotation to match the chord labels set used for the chord
recognition algorithm (i.e., DMaj6 → D); for this step you can choose any reduction strategy.

Once the function is defined in the notebook, test the function on the CSV file Beatles LetItBe.csv,
available in /data/csv/ folder, and print or plot the output.
Explain in the report each step of the preprocessing phase, focusing in particular on the reduction
strategy of the chord label set.
'''

import csv 
Fs = 22050
def chordsFromCsv(path):
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        chords = []
        i = 0;
        note = ['C','D','E','F','G','A','B']

        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                #print(math.trunc(float(row[1])*Fs), chords_WAV[math.trunc(float(row[1])*Fs)])
                print(math.trunc(float(row[1])*Fs))
                
                
                if "min" in row[2].split(':'):
                    temp = row[2].split(':')[0]+'m'
                else:
                    temp = row[2].split(':')[0]
                
                chords.append(temp.split('/')[0])

        for e in chords:
            if 'b' in e:
                chords[i]=note[note.index(chords[i][0])-1]+'#'
            i += 1
            
    return chords

audioCSV='data/csv/Beatles_LetItBe.csv' 
chords_CSV =chordsFromCsv(audioCSV);

print(chords_CSV)


40844
76172
104076
113036
131212
149388
185740
220812
256908
295308
331148
364428
391052
399756
417164
433548
468876
502668
537996
574092
608396
644236
670852
679420
695948
712076
748172
784780
819596
837516
855180
892812
928652
964748
1001612
1038220
1074316
1109388
1146764
1182604
1217676
1244604
1252887
1270156
1288588
1324428
1362060
1397388
1433484
1469836
1505932
1533051
1543052
1562252
1579404
1617036
1654412
1692300
1728396
1768588
1805708
1841804
1878668
1916556
1952396
1989260
2024588
2061964
2098828
2135180
2172812
2209420
2245772
2282636
2319500
2357900
2396044
2413964
2433932
2473100
2513036
2550924
2568588
2587788
2627212
2664332
2702988
2731961
2740620
2780044
2816652
2854284
2891660
2930828
2969228
3006860
3045516
3083660
3120780
3157900
3196044
3233676
3273100
3312268
3351180
3391116
3429772
3469196
3507852
3547276
3586444
3624588
3654134
3663244
3682021
3702156
3739532
3777420
3815564
3854220
3893644
3931443
3959716
3969932
3989132
4007052
4045452
4084108
4122508
4159

In [126]:
#Question 3
'''
Propose a metric for evaluating the template based chord recognition algorithm. 
A metric

m = f(λ_pred,λ_gt) (3)

is a scalar number that expresses how good is the algorithm in performing the task of chord recognition.
The proposed metric should have higher values when the chord recognition algorithm is able to predict
correctly the ground truth chords most of the times, lower values if the chord recognition algorithm often
fails at recognising the chords.

Write a function that takes as input the list of predicted chord labels, the list of ground truth chord
labels and computes the proposed metric value. The two input lists must have same length and the
output must be a scalar value.

Once the function is defined in the notebook, test the function on the two lists of predicted and ground
truth chord labels computed in Question 1 and Question 2.

In the report write a formal definition of the proposed metric and thoroughly explain the idea behind
the proposal. Can you imagine a musically informed strategy that weights differently mismatch errors of
the chord recognition algorithm?
'''

notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'];
trueChords=chordsFromCsv(audioCSV);

#recognizedChords=chordList(templates, template_sequence);
#da ultimare

recognizedChords=['C#', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'F', 'C', 'G', 'F', 'C', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'F', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'C', 'G', 'Am', 'A', 'F', 'F', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'Am', 'E', 'F', 'C', 'C', 'G', 'F', 'C', 'F', 'C', 'G', 'F', 'C']

'''
#Riduci la dimensione della lista del quesito 1
lenCSV = len(chords_CSV)
lenWAV = len(chords_WAV)
stepLenghts = int(round(lenWAV/lenCSV))
listNew = []
for i in range(0, lenWAV, stepLenghts):
        listNew.append(chords_WAV[i])
        
print(listNew)
print(len(listNew)==lenCSV)
'''

numChords=len(trueChords);
num_of_error=0;
imp_errors=[];

for chord in range(len(trueChords)):
    #print (chord,trueChords[chord])
    if(trueChords[chord]!=recognizedChords[chord]):
        num_of_error += 1;
        print(chord,trueChords[chord]);
        imp_errors.append(abs(notes.index(trueChords[chord])- notes.index(recognizedChords[chord])));
        #calcolare la quantita dell'errore

if (num_of_error!=0):
    perc_success1=math.trunc((100-((num_of_error/numChords)*100)));
    imp_errors_med=sum(imp_errors)/len(imp_errors);
    perc_success2=math.trunc((100-((imp_errors_med/11)*100)));
    print(perc_success1,perc_success2);
    
else:
    print(100);

40844
76172
104076
113036
131212
149388
185740
220812
256908
295308
331148
364428
391052
399756
417164
433548
468876
502668
537996
574092
608396
644236
670852
679420
695948
712076
748172
784780
819596
837516
855180
892812
928652
964748
1001612
1038220
1074316
1109388
1146764
1182604
1217676
1244604
1252887
1270156
1288588
1324428
1362060
1397388
1433484
1469836
1505932
1533051
1543052
1562252
1579404
1617036
1654412
1692300
1728396
1768588
1805708
1841804
1878668
1916556
1952396
1989260
2024588
2061964
2098828
2135180
2172812
2209420
2245772
2282636
2319500
2357900
2396044
2413964
2433932
2473100
2513036
2550924
2568588
2587788
2627212
2664332
2702988
2731961
2740620
2780044
2816652
2854284
2891660
2930828
2969228
3006860
3045516
3083660
3120780
3157900
3196044
3233676
3273100
3312268
3351180
3391116
3429772
3469196
3507852
3547276
3586444
3624588
3654134
3663244
3682021
3702156
3739532
3777420
3815564
3854220
3893644
3931443
3959716
3969932
3989132
4007052
4045452
4084108
4122508
4159

In [100]:
#Question 4
'''
Compute the proposed metric for the remaining 3 songs:
• audio Beatles HereComesTheSun.wav, CSV Beatles HereComesTheSun.csv
• audio Beatles PennyLane.wav, CSV Beatles PennyLane.csv
• audio Beatles ObLaDiObLaDa.wav, CSV Beatles ObLaDiObLaDa.csv
all contained in folders /data/wav/ and /data/csv/. Print or plot the metric values.
'''

'\nCompute the proposed metric for the remaining 3 songs:\n• audio Beatles HereComesTheSun.wav, CSV Beatles HereComesTheSun.csv\n• audio Beatles PennyLane.wav, CSV Beatles PennyLane.csv\n• audio Beatles ObLaDiObLaDa.wav, CSV Beatles ObLaDiObLaDa.csv\nall contained in folders /data/wav/ and /data/csv/. Print or plot the metric values.\n'

In [101]:
#Question 5
'''
Analyse how algorithm parameters affect the performance of the templated based chord recognition
algorithm.
Given one algorithm parameter (i.e., smoothing filter length L), choose a range of 3 possible values
for it (i.e.,L = [0, 10, 20] ). 
For each value of the parameter, compute the predicted labels and the correspondent metric value for each song.
This produces a list of 3 metric values for each song.

Plot the results for all songs in a figure where the x-axis corresponds to the parameter values, the
y-axis corresponds to metric values. An example of the plot is in Figure 1.

Repeat this for at least 3 different algorithm parameters. 
Note that in every experiment you need to change only one parameter value at a time, while the others must be fixed.

Show the 3 plots both in the Jupyter Notebook and in the report. 
What considerations can you do from the 3 plots that you have? 
Are some algorithm parameters affecting the results more than others?
Comment the results in the report.
'''

'\nAnalyse how algorithm parameters affect the performance of the templated based chord recognition\nalgorithm.\nGiven one algorithm parameter (i.e., smoothing filter length L), choose a range of 3 possible values\nfor it (i.e.,L = [0, 10, 20] ). \nFor each value of the parameter, compute the predicted labels and the correspondent metric value for each song.\nThis produces a list of 3 metric values for each song.\n\nPlot the results for all songs in a figure where the x-axis corresponds to the parameter values, the\ny-axis corresponds to metric values. An example of the plot is in Figure 1.\n\nRepeat this for at least 3 different algorithm parameters. \nNote that in every experiment you need to change only one parameter value at a time, while the others must be fixed.\n\nShow the 3 plots both in the Jupyter Notebook and in the report. \nWhat considerations can you do from the 3 plots that you have? \nAre some algorithm parameters affecting the results more than others?\nComment the res